### 1. Dataset

Samuel Singende
R204440W

In [ ]:
 file = open("/content/drive/MyDrive/shona_text.txt","r")
shona_text = file.read()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
shona_text

'\n   “Mwanangu ndiri kufara zvikuru kuona wave\nkukwanisa kudya sadza woga kudai, izvi\nzvinoratidza kuti wanaya, upenyu hwadzoka\nkwauri. Mapipi, akashanda basa guru kwazvo,\nndinotoshaya kuti ndomutenda nei. Ndakanga\nndarasikirwa newe, mwanakomana wangu,\nbaba vangu, chireramhuri. Chokwadi, ndanga\nndopishana mupfungwa kuti ko, ndakatadzei\nko kunaSamatenga, ndepapi pandakaposhera\nvadzimu vokwangu nevekwako.\n   “Zvandakabva kuno ndichiti ndoenda seri\nkwechikomo, kunaPiriviri, ndakaerera nzira yose\nmisodzi iri pamatama, vaipesana neni\nvaikahadzika zvakanyanya asi vachityira kure\nkubvunza chikonzero, vashoma vaive nezvivindi\nndivo vairovera moyo yavo pamabwe kwave\nkubvunza. Ipapo handaivanza kwete,\nndaivaudza kuti ndasiya mwanakomana\nwangu arere parupasa, achitandaza kwazvo.\nKo, ndaigovanzei izvo zvichinzi bundu rako\nwoga nderiri mumoyo, repaziso unosvinwa\nnavamwe?\n   “Chiona zvino nenji\nrandakasvikosanganidzana naro kwaMapipi?\nNdichiri chinhambwe toriro, ndakaona ach

In [ ]:
len(shona_text)

38458

In [ ]:
data = shona_text.lower()
print(data[500:550])

odzi iri pamatama, vaipesana neni
vaikahadzika zva


In [ ]:
import re
data = re.sub('https://\S+', '', data)
data = re.sub('www\S+', '', data)

In [ ]:
#REMOVING PUNCTUATIONS


punc = '''!()-[]{};:'"\,‘’“”<>./?@#$%^&*+_~→'''

for word in data:
  if word in punc:
    data = data.replace(word, "")

print(data)


   mwanangu ndiri kufara zvikuru kuona wave
kukwanisa kudya sadza woga kudai izvi
zvinoratidza kuti wanaya upenyu hwadzoka
kwauri mapipi akashanda basa guru kwazvo
ndinotoshaya kuti ndomutenda nei ndakanga
ndarasikirwa newe mwanakomana wangu
baba vangu chireramhuri chokwadi ndanga
ndopishana mupfungwa kuti ko ndakatadzei
ko kunasamatenga ndepapi pandakaposhera
vadzimu vokwangu nevekwako
   zvandakabva kuno ndichiti ndoenda seri
kwechikomo kunapiriviri ndakaerera nzira yose
misodzi iri pamatama vaipesana neni
vaikahadzika zvakanyanya asi vachityira kure
kubvunza chikonzero vashoma vaive nezvivindi
ndivo vairovera moyo yavo pamabwe kwave
kubvunza ipapo handaivanza kwete
ndaivaudza kuti ndasiya mwanakomana
wangu arere parupasa achitandaza kwazvo
ko ndaigovanzei izvo zvichinzi bundu rako
woga nderiri mumoyo repaziso unosvinwa
navamwe
   chiona zvino nenji
randakasvikosanganidzana naro kwamapipi
ndichiri chinhambwe toriro ndakaona achiita
kuhwitaira nekunyuruka akananga kwandiri
izvozvo zi

In [ ]:
# Removing the  words with digits

pattern = r'[0-9]'
data = re.sub(pattern, '', data)

print(data)


   mwanangu ndiri kufara zvikuru kuona wave
kukwanisa kudya sadza woga kudai izvi
zvinoratidza kuti wanaya upenyu hwadzoka
kwauri mapipi akashanda basa guru kwazvo
ndinotoshaya kuti ndomutenda nei ndakanga
ndarasikirwa newe mwanakomana wangu
baba vangu chireramhuri chokwadi ndanga
ndopishana mupfungwa kuti ko ndakatadzei
ko kunasamatenga ndepapi pandakaposhera
vadzimu vokwangu nevekwako
   zvandakabva kuno ndichiti ndoenda seri
kwechikomo kunapiriviri ndakaerera nzira yose
misodzi iri pamatama vaipesana neni
vaikahadzika zvakanyanya asi vachityira kure
kubvunza chikonzero vashoma vaive nezvivindi
ndivo vairovera moyo yavo pamabwe kwave
kubvunza ipapo handaivanza kwete
ndaivaudza kuti ndasiya mwanakomana
wangu arere parupasa achitandaza kwazvo
ko ndaigovanzei izvo zvichinzi bundu rako
woga nderiri mumoyo repaziso unosvinwa
navamwe
   chiona zvino nenji
randakasvikosanganidzana naro kwamapipi
ndichiri chinhambwe toriro ndakaona achiita
kuhwitaira nekunyuruka akananga kwandiri
izvozvo zi

### 2. Vocabulary

In [ ]:
from keras.preprocessing.text import Tokenizer

# Tokenize and preprocess
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index) + 1




### 3. Word Embeddings

In [ ]:
from gensim.models import Word2Vec

# Train word2vec model with gensim
sentences = [sentence.split() for sentence in shona_text.split('.')]
model_gensim = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model_gensim.save("word2vec.model")


### 4. RNN Models

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model1 = Sequential()
model1.add(Embedding(total_words, 100, input_length=5))
model1.add(Bidirectional(LSTM(150, return_sequences=True)))
model1.add(Dropout(0.2))
model1.add(LSTM(100))
model1.add(Dense(total_words, activation='softmax'))

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


##### With its own embedding layer:

##### With pre-trained embeddings:

In [ ]:
# Load the word2vec model
import numpy as np
model_gensim = Word2Vec.load("word2vec.model")
embedding_matrix = np.zeros((total_words, 100))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model_gensim.wv[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except KeyError:

        pass


model2 = Sequential()
model2.add(Embedding(total_words, 100, weights=[embedding_matrix], input_length=5, trainable=False))
model2.add(Bidirectional(LSTM(150, return_sequences=True)))
model2.add(Dropout(0.2))
model2.add(LSTM(100))
model2.add(Dense(total_words, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### 5. Training & Testing

In [6]:
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

input_sequences = []
for line in shona_text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=6, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=total_words)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

# Training Model 1
history1 = model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

# Training Model 2
history2 = model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

NameError: ignored

### Model Evaluation

In [ ]:
val_loss_model1 = history1.history['val_loss'][-1]
val_loss_model2 = history2.history['val_loss'][-1]

print(f"Validation Loss for Model 1: {val_loss_model1}")
print(f"Validation Loss for Model 2: {val_loss_model2}")


Validation Loss for Model 1: 10.468670845031738
Validation Loss for Model 2: 10.843488693237305


In [ ]:
if val_loss_model1 < val_loss_model2:
    best_model = model1
    best_model_name = "best_model1.h5"
else:
    best_model = model2
    best_model_name = "best_model2.h5"

best_model.save(best_model_name)
print(f"Saved the best model as {best_model_name}")


Saved the best model as best_model1.h5


In [ ]:
params_model1 = model1.count_params()
params_model2 = model2.count_params()

print(f"Model 1 has {params_model1} parameters.")
print(f"Model 2 has {params_model2} parameters.")


Model 1 has 879881 parameters.
Model 2 has 879881 parameters.


In [ ]:
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the previously saved model
model = load_model('best_model1.h5')

def predict_next_words(model, tokenizer, text, num_words=1):
    """
    Predict the next set of words using the trained model.

    Args:
    - model (keras.Model): The trained model.
    - tokenizer (Tokenizer): The tokenizer object used for preprocessing.
    - text (str): The input text.
    - num_words (int): The number of words to predict.

    Returns:
    - str: The predicted words.
    """
    for _ in range(num_words):
        # Tokenize and pad the text
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=5, padding='pre')

        # Predict the next word
        predicted_probs = model.predict(sequence, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)

        # Convert the predicted word index to a word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        # Append the predicted word to the text
        text += " " + output_word

    return ' '.join(text.split(' ')[-num_words:])


# Prompt the user for input
user_input = input("Please type five words in Shona: ")

# Predict the next words
predicted_words = predict_next_words(model, tokenizer, user_input, num_words=3)
print(f"The next words might be: {predicted_words}")


Please type five words in Shona: Vakapedzisira kudaro vomusairira pamwe nekumudinhidza
The next words might be: zvayo asiya yevarwere
